In [3]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")

if not SUPABASE_URL or not SUPABASE_SERVICE_ROLE_KEY:
    raise RuntimeError("Missing SUPABASE_URL or SUPABASE_SERVICE_ROLE_KEY")

# Supabase REST API config
REST_URL = SUPABASE_URL.rstrip("/") + "/rest/v1/podcast_profiles"
HEADERS = {
    "apikey": SUPABASE_SERVICE_ROLE_KEY,
    "Authorization": f"Bearer {SUPABASE_SERVICE_ROLE_KEY}",
}

# Fetch all podcast_profiles from Supabase
rows = []
page_size = 1000
start = 0

print("Fetching podcast_profiles from Supabase...")
while True:
    resp = requests.get(
        REST_URL,
        headers=HEADERS,
        params={"limit": page_size, "offset": start},
        timeout=60,
    )
    if resp.status_code not in (200, 206):
        raise RuntimeError(f"Failed to fetch podcast_profiles: HTTP {resp.status_code} - {resp.text}")
    
    batch = resp.json()
    if not batch:
        break
    
    print(f"Fetched {len(batch)} rows (total: {len(rows) + len(batch)})")
    rows.extend(batch)
    
    if len(batch) < page_size:
        break
    
    start += page_size

print(f"\nTotal rows fetched: {len(rows)}")

# Convert to pandas DataFrame
df = pd.DataFrame(rows)
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()



Fetching podcast_profiles from Supabase...
Fetched 1000 rows (total: 1000)
Fetched 1000 rows (total: 2000)
Fetched 400 rows (total: 2400)

Total rows fetched: 2400

DataFrame shape: (2400, 12)

Columns: ['id', 'rss_feed_url', 'podcast_name', 'supplier_id', 'supplier_name', 'network_id', 'network_name', 'genre', 'status_code', 'created_at', 'updated_at', 'RSS_request_status_code']

First few rows:


,id,rss_feed_url,podcast_name,supplier_id,supplier_name,network_id,network_name,genre,status_code,created_at,updated_at,RSS_request_status_code
0,878a543f-5496-45e4-ae18-08da752e0cf7,https://feed.pod.space/45minuteravalversjovide...,45 minuter AV,b3222a28-d195-433c-d8f1-08da752edf7e,Podspace,None,,Mat/Dryck,200,2025-10-28T22:22:29.23343+00:00,2025-10-29T01:19:38.344178+00:00,200
1,e513be13-9f80-4c3c-4ecd-08dc8570d9ab,https://feed.pod.space/mianochsarasskrivarlya,Mian och Saras skrivarlya,b3222a28-d195-433c-d8f1-08da752edf7e,Podspace,None,,Utbildning,200,2025-10-28T22:40:43.644234+00:00,2025-10-29T01:19:41.390469+00:00,200
2,5762bee5-4e79-4bfa-8837-08dc89033779,https://feeds.acast.com/public/shows/665daacf4...,2 Under 2,4547b65d-ccf7-469c-d8ec-08da752edf7e,Acast,None,,Barn/Familj,200,2025-10-28T22:22:24.275026+00:00,2025-10-29T01:19:45.515727+00:00,200
3,b97ebc42-948e-4988-ab11-08da752e0cf7,https://feeds.acast.com/public/shows/c13451b2-...,50 nyanser av pengar,4547b65d-ccf7-469c-d8ec-08da752edf7e,Acast,None,,Privatekonomi,200,2025-10-28T22:22:30.092405+00:00,2025-10-29T01:19:48.933804+00:00,200
4,8e68958e-186a-43cb-ad21-08da752e0cf7,https://api.sr.se/api/rss/pod/33855,Mia Blomgren,0d60154d-4998-47b5-d8eb-08da752edf7e,Sveriges Radio,30efa239-0cd5-48b6-378b-08da752edfd0,Sveriges Radio,Nyheter/Politik,200,2025-10-28T22:40:43.265919+00:00,2025-10-29T01:19:58.792341+00:00,200


In [14]:
# Show the distribution of status_code
print("\n\nstatus_code Distribution:")
print("=" * 80)
status_code_counts = df['status_code'].value_counts(dropna=False)
for status, count in status_code_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{str(status):20} {count:5} ({percentage:5.2f}%)")

print(f"\n\nTotal unique status_code: {df['status_code'].nunique(dropna=True)}")
print(f"Null/Missing status_code: {df['status_code'].isna().sum()}")




status_code Distribution:
200                   2377 (99.04%)
404                     19 ( 0.79%)
400                      1 ( 0.04%)
410                      1 ( 0.04%)
500                      1 ( 0.04%)
503                      1 ( 0.04%)


Total unique status_code: 6
Null/Missing status_code: 0


In [12]:
# Show the distribution of RSS_request_status_code
print("\n\nRSS_request_status_code Distribution:")
print("=" * 80)
rss_status_counts = df['RSS_request_status_code'].value_counts(dropna=False)
for status, count in rss_status_counts.items():
    percentage = (count / len(df)) * 100
    print(f"{str(status):20} {count:5} ({percentage:5.2f}%)")

print(f"\n\nTotal unique RSS_request_status_code: {df['RSS_request_status_code'].nunique(dropna=True)}")
print(f"Null/Missing RSS_request_status_code: {df['RSS_request_status_code'].isna().sum()}")




RSS_request_status_code Distribution:
200                   2315 (96.46%)
404                     77 ( 3.21%)
400                      2 ( 0.08%)
410                      2 ( 0.08%)
500                      2 ( 0.08%)
503                      1 ( 0.04%)
0                        1 ( 0.04%)


Total unique RSS_request_status_code: 7
Null/Missing RSS_request_status_code: 0


TODO: Some talk about the statuscode ratio.

In [6]:
from urllib.parse import urlparse

# Extract base URLs from RSS feed URLs
def extract_base_url(rss_url):
    """Extract base URL (scheme + netloc) from RSS feed URL"""
    if pd.isna(rss_url) or not rss_url:
        return None
    try:
        parsed = urlparse(rss_url)
        return f"{parsed.scheme}://{parsed.netloc}"
    except:
        return None

df['base_url'] = df['rss_feed_url'].apply(extract_base_url)

# Show unique base URLs and their counts
base_url_counts = df['base_url'].value_counts()




# Show the distribution
print("\n\nBase URL Distribution:")
print("=" * 80)
for base_url, count in base_url_counts.head(20).items():
    percentage = (count / len(df)) * 100
    print(f"{base_url:50} {count:5} ({percentage:5.2f}%)")


print(f"\n\nTotal unique base URLs: {len(base_url_counts)}")
print(f"Null/Missing RSS URLs: {df['rss_feed_url'].isna().sum()}")



Base URL Distribution:
https://feeds.acast.com                             1136 (47.33%)
https://feed.pod.space                               521 (21.71%)
https://api.sr.se                                    272 (11.33%)
https://rss.podplaystudio.com                        150 ( 6.25%)
https://rss.acast.com                                109 ( 4.54%)
https://podcast.stream.schibsted.media                94 ( 3.92%)
https://access.acast.com                              70 ( 2.92%)
https://feed.khz.se                                   20 ( 0.83%)
https://cdn.radioplay.se                              12 ( 0.50%)
http://www.ilikeradio.se                               8 ( 0.33%)
https://www.ilikeradio.se                              8 ( 0.33%)


Total unique base URLs: 11
Null/Missing RSS URLs: 0


In [8]:
# Show the distribution of suppliername
print("\n\nSupplier Name Distribution:")
print("=" * 80)
supplier_counts = df['supplier_name'].value_counts(dropna=False)
for supplier, count in supplier_counts.head(20).items():
    percentage = (count / len(df)) * 100
    print(f"{str(supplier):40} {count:5} ({percentage:5.2f}%)")

print(f"\n\nTotal unique supplier_name: {df['supplier_name'].nunique(dropna=True)}")
print(f"Null/Missing supplier_name: {df['supplier_name'].isna().sum()}")





Supplier Name Distribution:
Acast                                     1419 (59.13%)
Podspace                                   520 (21.67%)
Sveriges Radio                             272 (11.33%)
Podplay                                    155 ( 6.46%)
Viaplay Group Radio                         34 ( 1.42%)


Total unique supplier_name: 5
Null/Missing supplier_name: 0


TODO: dubbelcheck - distro text 

In [10]:

print("\n\network_name Distribution:")
print("=" * 80)
network_counts = df['network_name'].value_counts(dropna=False)
for network_name, count in network_counts.head(20).items():
    percentage = (count / len(df)) * 100
    print(f"{str(network_name):40} {count:5} ({percentage:5.2f}%)")
print(f"\n\nTotal unique network_name: {df['network_name'].nunique(dropna=True)}")
print(f"Null/Missing network_name: {df['network_name'].isna().sum()}")




etwork_name Distribution:
                                          1749 (72.88%)
Sveriges Radio                             272 (11.33%)
Bonnier News                               116 ( 4.83%)
Bauer Media                                113 ( 4.71%)
Schibsted Marketing Services (network)      87 ( 3.62%)
Perfect Day Media                           45 ( 1.88%)
Polpo Play AB                               11 ( 0.46%)
Make Sense Media                             6 ( 0.25%)
Ek & Borg Productions AB                     1 ( 0.04%)


Total unique network_name: 9
Null/Missing network_name: 0


In [11]:
# Show the distribution of genre
print("\n\nGenre Distribution:")
print("=" * 80)
genre_counts = df['genre'].value_counts(dropna=False)
for genre, count in genre_counts.head(20).items():
    percentage = (count / len(df)) * 100
    print(f"{str(genre):40} {count:5} ({percentage:5.2f}%)")

print(f"\n\nTotal unique genre: {df['genre'].nunique(dropna=True)}")
print(f"Null/Missing genre: {df['genre'].isna().sum()}")




Genre Distribution:
Underhållning                              787 (32.79%)
Samhälle/Dokumentär                        336 (14.00%)
Sport                                      253 (10.54%)
Intervjuer                                 131 ( 5.46%)
Träning/Hälsa                              113 ( 4.71%)
Thriller/Crime                             111 ( 4.62%)
Nyheter/Politik                            100 ( 4.17%)
Näringsliv                                  93 ( 3.88%)
Kultur                                      74 ( 3.08%)
Musik                                       71 ( 2.96%)
Barn/Familj                                 55 ( 2.29%)
Mat/Dryck                                   47 ( 1.96%)
Privatekonomi                               39 ( 1.62%)
Religion/Andlighet                          34 ( 1.42%)
Skönhet/Mode                                28 ( 1.17%)
Hem/Inredning/Trädgård                      26 ( 1.08%)
Resor                                       25 ( 1.04%)
Teknologi/Vetenskap       